In [1]:
import pandas as pd
import altair as alt
import numpy as np


In [2]:
df = pd.read_csv('https://projects.fivethirtyeight.com/nfl-api/nfl_elo.csv')

In [3]:
df.columns

Index(['date', 'season', 'neutral', 'playoff', 'team1', 'team2', 'elo1_pre',
       'elo2_pre', 'elo_prob1', 'elo_prob2', 'elo1_post', 'elo2_post',
       'qbelo1_pre', 'qbelo2_pre', 'qb1', 'qb2', 'qb1_value_pre',
       'qb2_value_pre', 'qb1_adj', 'qb2_adj', 'qbelo_prob1', 'qbelo_prob2',
       'qb1_game_value', 'qb2_game_value', 'qb1_value_post', 'qb2_value_post',
       'qbelo1_post', 'qbelo2_post', 'score1', 'score2'],
      dtype='object')

In [4]:
df_xs = df[['season', 'neutral', 'playoff', 'elo1_pre', 'elo2_pre', 'elo_prob1', 'elo_prob2', 'score1', 'score2']]
df_xs

season  neutral playoff     elo1_pre     elo2_pre  elo_prob1  \
0        1920        0     NaN  1503.947000  1300.000000   0.824651   
1        1920        0     NaN  1493.002000  1504.908000   0.575819   
2        1920        0     NaN  1516.108000  1478.004000   0.644171   
3        1920        0     NaN  1368.333000  1300.000000   0.682986   
4        1920        0     NaN  1504.688000  1300.000000   0.825267   
...       ...      ...     ...          ...          ...        ...   
16536    2019        0       d  1694.572097  1552.346950   0.767256   
16537    2019        0       d  1631.858399  1579.312894   0.662990   
16538    2019        0       c  1707.523977  1662.579362   0.653145   
16539    2019        0       c  1651.872498  1643.322690   0.604293   
16540    2019        1       s          NaN          NaN        NaN   

       elo_prob2  score1  score2  
0       0.175349    48.0     0.0  
1       0.424181    14.0     0.0  
2       0.355829    45.0     0.0  
3       0.317014    20.0     0.0  
4       0.174733    48.0     0.0  
...          ...     ...     ...  
16536   0.232744    51.0    31.0  
16537   0.337010    28.0    23.0  
16538   0.346855     NaN     NaN  
16539   0.395707     NaN     NaN  
16540        NaN     NaN     NaN  

[16541 rows x 9 columns]

In [5]:
df_xs_reg = df_xs[df_xs['playoff'].isna()]
df_xs_reg['elo_diff'] = df_xs_reg['elo1_pre'] - df_xs_reg['elo2_pre']
df_xs_reg['score_diff'] = df_xs_reg['score1'] - df_xs_reg['score2']
df_xs_reg.head(3)

/home/simardeep/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/simardeep/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


season  neutral playoff  elo1_pre  elo2_pre  elo_prob1  elo_prob2  score1  \
0    1920        0     NaN  1503.947  1300.000   0.824651   0.175349    48.0   
1    1920        0     NaN  1493.002  1504.908   0.575819   0.424181    14.0   
2    1920        0     NaN  1516.108  1478.004   0.644171   0.355829    45.0   

   score2  elo_diff  score_diff  
0     0.0   203.947        48.0  
1     0.0   -11.906        14.0  
2     0.0    38.104        45.0

In [6]:
alt.data_transformers.disable_max_rows()

base = alt.Chart(df_xs_reg).mark_circle(opacity = 0.5).encode(
    alt.X('elo_diff:Q', title = 'Difference in elo (home - away)'),
    alt.Y('score_diff:Q', title = 'Difference in scores (home - away)')
).properties(
    title = 'Diff in score vs diff in elo for season games',# hopefully I call this right
    height = 600,
    width = 800
)

order = 1

lin_fit = base.transform_regression(
    'elo_diff', 'score_diff', method = 'poly', order = order, as_ = ['elo_diff', str(order)]
).mark_line(color = 'red').transform_fold(
    [str(order)], as_ = ['degree', 'score_diff']
).encode()

chart = alt.layer(base, lin_fit)
chart

AttributeError: 'Chart' object has no attribute 'transform_regression'

In [ ]:
chart.save('img/score_vs_elo.html')